In [1]:
import pandas as pd
# events_df = pd.read_csv("https://api.mockaroo.com/api/3f045400?count=1000&key=9aadb790")
events_df = pd.read_csv("../events.csv")

In [2]:
event_tags_list = [value if isinstance(value, str) else '' for value in events_df['tags']]
event_name_list = [value if isinstance(value, str) else '' for value in events_df['event_name']]
event_location_list = [value if isinstance(value, str) else '' for value in events_df['location']]
event_description_list = [value if isinstance(value, str) else '' for value in events_df['description']]
print(event_tags_list)
print(event_name_list)
print(event_location_list)
print(event_description_list)

['Tradeshow, Auto & Automotive, Rubber & Tyres', 'Tradeshow, Building & Construction', 'Conference, Medical & Pharma, Investment', 'Tradeshow, IT & Technology', 'Tradeshow, Travel & Tourism', 'Tradeshow, Electric & Electronics', 'Tradeshow, Renewable Energy, Power & Energy', 'Tradeshow, Food & Beverages, Bakery & Confectionery', 'Conference, Science & Research, IT & Technology', 'Conference, Science & Research, Education & Training, Economics', 'Tradeshow, Industrial Engineering, Asset Management', 'Conference, Business Services', 'Conference, Science & Research, Education & Training, Economics', 'Conference, Medical & Pharma, Education & Training', 'Conference, Business Services', 'Conference, Business Services, IT & Technology, Artificial Intelligence, Big Data, Marketing', 'Tradeshow, Food & Beverages, Bakery & Confectionery, Tea & Coffee, Wine & Spirits, Hotel, Restaurant, Hospitality', 'Tradeshow, Building & Construction, Home & Office, Interior Design', 'Conference, Education & T

In [76]:
import tensorflow as tf
import numpy as np
def get_recommendations_tfidf(query, data, top_n=1, output_mode='tf_idf'):
    # Create the TextVectorization layer
    vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=None,  # Default value, no maximum limit on vocabulary size
        output_mode=output_mode
    )

    data = list(set(data))
    vectorizer.adapt(data)
    X = vectorizer(data+[query])
    # Compute the pairwise cosine similarity
    cosine_sim = tf.linalg.matmul(X, X, transpose_b=True)

    # Get the pairwise cosine similarity scores for the document
    sim_scores = list(enumerate(cosine_sim[-1, :-1]))

    # Sort the documents by similarity score in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top n recommendations
    top_docs = sim_scores[:top_n]
    # Return the indices and recommendations
    output = {}
    c=0
    for i, j in top_docs:
        match_score = float(j)
        matched_events = events_df[
            (events_df['tags']==data[i]) |
            (events_df['event_name']==data[i]) |
            (events_df['location']==data[i]) |
            (events_df['description']==data[i])
            ]
        event_id_list = [value if not np.isnan(value) else '' for value in matched_events['id']]
        event_tags_list = [value if isinstance(value, str) else '' for value in matched_events['tags']]
        event_name_list = [value if isinstance(value, str) else '' for value in matched_events['event_name']]
        event_location_list = [value if isinstance(value, str) else '' for value in matched_events['location']]
        event_description_list = [value if isinstance(value, str) else '' for value in matched_events['description']]
        for index in range(len(matched_events)):
            if c == 10:
                break
            output[c] = {
                'event_id': event_id_list[index],
                'event_name': event_name_list[index],
                'location': event_location_list[index],
                'description': event_description_list[index],
                'tags': event_tags_list[index],
                'match_score': match_score
            }
            c += 1
    return output

In [77]:
query = "digital tech startup"
data = event_tags_list + event_name_list + event_description_list + event_location_list

for output_mode in ['tf_idf', 'int', 'count', 'multi_hot']:
    recommendations = get_recommendations_tfidf(query, data, top_n=5)
    for out in recommendations.values():
        print(out)
    print()

{'event_id': 116, 'event_name': 'Digital Transformation Indonesia Conference and Expo (DTI-CX)', 'location': 'Jakarta', 'description': 'Digital Transformation Indonesia Conference and Expo bring together corporates, SOEs, and governments who are looking for tech partners to learn and execute their digital transformation plans in the future...', 'tags': 'Tradeshow, IT & Technology', 'match_score': 70.08061218261719}
{'event_id': 103, 'event_name': 'Seafood Show of Asia Expo', 'location': 'Jakarta', 'description': "Int'l expo on fish and seafood  products, materials,  processing, tech and packaging in indonesia.", 'tags': 'Tradeshow, Agriculture & Forestry, Meat, Poultry & Seafood, Food & Beverages', 'match_score': 31.977052688598633}
{'event_id': 54, 'event_name': 'Indonesia Digital Technology Expo (INDITEX)', 'location': 'East Bekasi', 'description': 'An event that brought together business Digital Lifestyle technology and innovation Collide', 'tags': 'Tradeshow, Business Services, IT 